In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing

In [2]:
import tensorflow_datasets as tfds

max_len = 200
n_words = 10000
dim_embedding = 256
EPOCHS = 20
BATCH_SIZE = 500

def load_data():
   # 데이터 로드
   (X_train, y_train),(X_test,y_test) = datasets.imdb.load_data(num_words = n_words)
   #문장을 max_len 이 되도록 채워 넣는다
   X_train = preprocessing.sequence.pad_sequences(X_train,maxlen=max_len)
   X_test = preprocessing.sequence.pad_sequences(X_test,maxlen=max_len)
   return (X_train, y_train),(X_test, y_test)

In [11]:
def build_model():
  model = models.Sequential()
  #입력 : -eEmbedding Layer.
  #모델은 크기의 정수 행렬을 입력으로 취한다(batch, input_length).
  #모델의 출력은 차원이다 (input_length, dim_embedding).
  #입력 중 가장 큰 정수는 n_words 보다 작거나 같다 (어휘 크기).
  model.add(layers.Embedding(n_words,
                              dim_embedding, input_length=max_len))

  model.add(layers.Dropout(0.3))

  #각 n_words 특징에서 특징 벡터의 최댓값을 취함
  model.add(layers.GlobalMaxPool1D())
  model.add(layers.Dense(128,activation = 'relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  return model

In [12]:
(X_train,y_train),(X_test,y_test)=load_data()
model = build_model()

In [13]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 256)          2560000   
                                                                 
 dropout_2 (Dropout)         (None, 200, 256)          0         
                                                                 
 global_max_pooling1d_2 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                      

In [14]:
model.compile(optimizer="adam", loss= "binary_crossentropy",
              metrics =["accuracy"]
              )

In [15]:
score = model.fit(X_train, y_train,
                  epochs=EPOCHS,
                  batch_size= BATCH_SIZE,
                  validation_data =(X_test,y_test))

Epoch 1/20
50/50 [==============================] - 33s 644ms/step - loss: 0.6743 - accuracy: 0.6290 - val_loss: 0.6359 - val_accuracy: 0.8350
Epoch 2/20
50/50 [==============================] - 32s 635ms/step - loss: 0.4679 - accuracy: 0.8366 - val_loss: 0.3652 - val_accuracy: 0.8590
Epoch 3/20
50/50 [==============================] - 32s 642ms/step - loss: 0.2823 - accuracy: 0.8837 - val_loss: 0.3053 - val_accuracy: 0.8747
Epoch 4/20
50/50 [==============================] - 30s 597ms/step - loss: 0.2174 - accuracy: 0.9158 - val_loss: 0.2904 - val_accuracy: 0.8784
Epoch 5/20
50/50 [==============================] - 37s 730ms/step - loss: 0.1698 - accuracy: 0.9378 - val_loss: 0.2908 - val_accuracy: 0.8764
Epoch 6/20
50/50 [==============================] - 30s 610ms/step - loss: 0.1333 - accuracy: 0.9566 - val_loss: 0.2976 - val_accuracy: 0.8738
Epoch 7/20
50/50 [==============================] - 31s 633ms/step - loss: 0.1029 - accuracy: 0.9683 - val_loss: 0.3060 - val_accuracy: 0.8695

In [18]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('test accuracy:',score[1])

50/50 [==============================] - 4s 86ms/step - loss: 0.4965 - accuracy: 0.8514

Test score: 0.4964689314365387
test accuracy: 0.8514400124549866


In [24]:
predictions = model.predict(X_test)
print(predictions)

782/782 [==============================] - 4s 5ms/step
[[0.00405705]
 [0.99937564]
 [0.13621897]
 ...
 [0.00331138]
 [0.00128106]
 [0.9153306 ]]
